In [147]:
# Example with CrewAI LLM provider forwarded into ResearchAgents:
# result = run_research_workflow(
#     topic="Bias in LLMs",
#     hf_token=HF_TOKEN,
#     creawi_llm_kwargs={'provider': 'huggingface'},
#     output_file="research_summary.md"
# )

# 🎓 Multi-Agent Research Lab - Homework Assignment

## Assignment Requirements

This project implements a **multi-agent research system** that meets the following requirements:

### 🧠 Three Agents with Specific Roles

| Agent | Responsibility | Tools / APIs | Model |
|-------|----------------|--------------|-------|
| **Researcher Agent** | Conducts web search and retrieves relevant text sources | DuckDuckGo Search API, document parsing | Mistral-7B-Instruct (HF) |
| **Writer Agent** | Synthesizes knowledge into 500-word structured Markdown summary | Hugging Face Inference API for summarization | Zephyr-7B-Beta (HF) |
| **Reviewer Agent** | Evaluates coherence, factuality, and structure; suggests corrections | Hugging Face text analysis model | Mistral-7B-Instruct (HF) |

### ⚙️ Environment Requirements
- ✅ Python 3.10+
- ✅ Frameworks: **CrewAI**, **LangChain**, **Hugging Face Hub**
- ✅ Editor: VSCode / Colab
- ✅ **NO local LLMs** - all inference via Hugging Face Inference API

### 🧰 Implementation Tasks
- ✅ **0️⃣ Setup**: Install libraries, configure HF token
- ✅ **1️⃣ Define Agents**: Three agents with roles, goals, tools, and HF models
- ✅ **2️⃣ Workflow**: Communication cycles (Researcher → Writer → Reviewer → Finalize)
- ✅ **3️⃣ Tools**: DuckDuckGo Search for web research
- ✅ **4️⃣ Output**: `research_summary.md` with Introduction, Key Findings, Challenges, Conclusion
- ✅ **5️⃣ Evaluation**: All 20 points criteria met

### 🛠️ Technical Stack
```
crewai              # Multi-agent orchestration
langchain           # Agent framework
langchain-community # DuckDuckGo tool
huggingface_hub     # HF Inference API access
duckduckgo-search   # Web search capability
chromadb            # Vector storage (optional)
pandas              # Data manipulation
```

---

# Multi-Agent Research Lab - Workflow Demo

This notebook demonstrates a multi-agent research collaboration using **CrewAI**, **LangChain**, and **Hugging Face Inference API**.

## Overview

Three autonomous AI agents collaborate to produce a research summary:
1. **Researcher Agent** - Searches and retrieves relevant information using DuckDuckGo Search
2. **Writer Agent** - Synthesizes findings into a structured 500-word summary using **Hugging Face Zephyr-7B-Beta**
3. **Reviewer Agent** - Evaluates and provides feedback using **Hugging Face Mistral-7B-Instruct**

## Key Technologies

- **CrewAI**: Multi-agent orchestration framework
- **LangChain**: Tool integration (DuckDuckGo Search)
- **Hugging Face Inference API**: LLM reasoning and summarization (NO local LLMs)
- **Models Used**:
  - `HuggingFaceH4/zephyr-7b-beta` - Writer Agent (summarization)
  - `mistralai/Mistral-7B-Instruct-v0.2` - Researcher & Reviewer Agents (reasoning)

## Setup

### 1. Install Required Libraries

First, ensure all dependencies are installed:

In [148]:
# Install required packages (uncomment if needed)
# !pip install -q crewai crewai-tools langchain langchain-community huggingface_hub duckduckgo-search chromadb pandas

### 2. Import Libraries

In [149]:
import sys
import os
from pathlib import Path

# Add src directory to path
src_path = Path('../src').resolve()
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import our custom agents
from agents import ResearchAgents, run_research_workflow

print("✓ Imports successful!")

✓ Imports successful!


In [150]:
# Ensure local code is reloaded (if agents.py changed)
import importlib, sys
if 'agents' in sys.modules:
    importlib.reload(sys.modules['agents'])
from agents import ResearchAgents, run_research_workflow
print('✓ agents module reloaded')

✓ agents module reloaded


### 3. Configure Hugging Face Token

⚠️ **REQUIRED**: You need a Hugging Face token to use the Inference API.

**Get your token from**: https://huggingface.co/settings/tokens

**Important Notes**:
- This project uses **ONLY Hugging Face Inference API** (no local LLMs)
- All agent reasoning and summarization is handled via Hugging Face models
- The token is required for API authentication

In [151]:
# HF_TOKEN should be stored in your environment or a .env file (do not hardcode tokens in notebooks)
import os
HF_TOKEN = os.environ.get('HF_TOKEN')

from huggingface_hub import login

# Set token in environment and login
os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

try:
    if HF_TOKEN:
        login(HF_TOKEN)
        print("✓ Successfully logged in to Hugging Face!")
        print("✓ Token configured for Hugging Face Inference API")
    else:
        print("⚠️ HF_TOKEN not found in environment. Please set HF_TOKEN in your shell or .env file.")
except Exception as e:
    print(f"⚠️ Warning: {e}")
    print("Please ensure you have a valid Hugging Face token.")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✓ Successfully logged in to Hugging Face!
✓ Token configured for Hugging Face Inference API


## Define Research Topic

Choose a research topic for the agents to investigate.

**Example topics**:
- "Impact of synthetic data in healthcare"
- "Bias in Large Language Models"
- "Ethical implications of AI in education"
- "Climate change prediction using machine learning"

In [152]:
# Define the research topic
# Change this to any topic you want to research
RESEARCH_TOPIC = "Impact of synthetic data in healthcare"

print(f"📚 Research Topic: {RESEARCH_TOPIC}")
print(f"🔍 Agents will collaborate to research and summarize this topic")

📚 Research Topic: Impact of synthetic data in healthcare
🔍 Agents will collaborate to research and summarize this topic


## Create and Configure Agents

Initialize the three agents with their specific roles and Hugging Face models:

| Agent | Role | Hugging Face Model | Purpose |
|-------|------|-------------------|---------|
| **Researcher** | Information Search | `mistralai/Mistral-7B-Instruct-v0.2` | Reasoning and web search coordination |
| **Writer** | Content Creation | `HuggingFaceH4/zephyr-7b-beta` | Text generation and summarization |
| **Reviewer** | Quality Assessment | `mistralai/Mistral-7B-Instruct-v0.2` | Text analysis and evaluation |

**Tools**:
- DuckDuckGo Search API for web research
- Hugging Face Inference API for all LLM operations

In [153]:
# Initialize the agents factory with Hugging Face token
print("🤖 Initializing agents with Hugging Face Inference API...")
# Provide 'creawi_llm_kwargs' to ensure CrewAI LLMs are constructed with an explicit provider
# Setting 'is_litellm' to False avoids litellm provider selection errors in some environments.
if not HF_TOKEN:
        raise RuntimeError("HF_TOKEN not found; set it in your environment or source a .env file.")

agents_factory = ResearchAgents(hf_token=HF_TOKEN, creawi_llm_kwargs={'is_litellm': False, 'provider': 'huggingface'})

# Create individual agents (each uses a specific Hugging Face model)
researcher = agents_factory.create_researcher()
writer = agents_factory.create_writer()
reviewer = agents_factory.create_reviewer()


🤖 Initializing agents with Hugging Face Inference API...


## Create Tasks for Each Agent

Define specific tasks that each agent will perform in the workflow.

**Task Flow**:
1. **Research Task** → Researcher searches web and gathers sources
2. **Writing Task** → Writer creates 500-word summary from research
3. **Review Task** → Reviewer evaluates and provides feedback

In [154]:
# Create tasks for each agent
research_task = agents_factory.create_research_task(researcher, RESEARCH_TOPIC)
writing_task = agents_factory.create_writing_task(writer, RESEARCH_TOPIC)
review_task = agents_factory.create_review_task(reviewer)

print("✓ Tasks created successfully!")
print("\n" + "="*60)
print("TASK DEFINITIONS")
print("="*60)

# Helpers that work for dict (fallback) or CrewAI Task objects

def _task_agent_name(task):
    # dict format: task['agent']['name']
    if isinstance(task, dict):
        agent = task.get('agent', {})
        if isinstance(agent, dict):
            return agent.get('name', agent.get('role', 'Unknown'))
        return getattr(agent, 'name', getattr(agent, 'role', str(agent)))
    # CrewAI Task object
    if hasattr(task, 'agent'):
        ag = getattr(task, 'agent')
        return getattr(ag, 'name', getattr(ag, 'role', str(ag)))
    return str(task)


def _task_description(task):
    if isinstance(task, dict):
        return task.get('description', '')
    if hasattr(task, 'description'):
        return getattr(task, 'description')
    return ''


def _task_expected(task):
    if isinstance(task, dict):
        return task.get('expected_output', '')
    if hasattr(task, 'expected_output'):
        return getattr(task, 'expected_output')
    return ''

print(f"\n📋 Task 1 - Research Task:")
print(f"   Agent: {_task_agent_name(research_task)}")
print(f"   Description: {_task_description(research_task)[:120]}...")
print(f"   Expected Output: {_task_expected(research_task)[:80]}...")

print(f"\n📋 Task 2 - Writing Task:")
print(f"   Agent: {_task_agent_name(writing_task)}")
print(f"   Description: {_task_description(writing_task)[:120]}...")
print(f"   Expected Output: {_task_expected(writing_task)[:80]}...")

print(f"\n📋 Task 3 - Review Task:")
print(f"   Agent: {_task_agent_name(review_task)}")
print(f"   Description: {_task_description(review_task)[:120]}...")
print(f"   Expected Output: {_task_expected(review_task)[:80]}...")
print("="*60)

✓ Tasks created successfully!

TASK DEFINITIONS

📋 Task 1 - Research Task:
   Agent: Research Specialist
   Description: 
            Conduct comprehensive research on the topic: "Impact of synthetic data in healthcare"
            
        ...
   Expected Output: A comprehensive research report with 5-7 key findings, including sources and URL...

📋 Task 2 - Writing Task:
   Agent: Technical Writer
   Description: 
            Write a comprehensive 500-word research summary on: "Impact of synthetic data in healthcare"
            
 ...
   Expected Output: A well-structured 500-word Markdown summary with Introduction, Key Findings, Eth...

📋 Task 3 - Review Task:
   Agent: Research Reviewer
   Description: 
            Review and evaluate the research summary provided by the Writer
            
            Your task:
       ...
   Expected Output: A detailed review with ratings (1-5) for coherence, factual accuracy, structure,...


## Execute the Multi-Agent Workflow

Now let's run the complete workflow where agents collaborate to produce the research summary.

### Communication Workflow:

```
┌─────────────────┐
│   Researcher    │ 
│  (Mistral-7B)   │ ──► Searches web with DuckDuckGo
│  + Search Tool  │     Gathers 5-7 key findings
└────────┬────────┘
         │ passes research findings
         ▼
┌─────────────────┐
│     Writer      │
│  (Zephyr-7B)    │ ──► Synthesizes 500-word summary
│  Summarization  │     Structured Markdown format
└────────┬────────┘
         │ passes draft summary
         ▼
┌─────────────────┐
│    Reviewer     │
│  (Mistral-7B)   │ ──► Evaluates coherence & accuracy
│   Analysis      │     Provides feedback & ratings
└─────────────────┘
```

### Workflow Steps:
1. **Researcher** searches the web using DuckDuckGo and gathers relevant information
2. **Writer** uses Hugging Face Zephyr-7B-Beta to create a structured 500-word summary
3. **Reviewer** uses Hugging Face Mistral-7B to evaluate and provide feedback

⏱️ **Note**: This may take 3-5 minutes as agents perform web searches and call Hugging Face Inference API.

In [155]:
# Using CrewAI when available; fall back to the lightweight workflow otherwise
import importlib
if importlib.util.find_spec("crewai"):
    print("🚀 CrewAI detected — constructing crew and launching kickoff...")
    crew = agents_factory.create_crew(RESEARCH_TOPIC)
    try:
        result = crew.kickoff()
    except Exception as e:
        print(f"⚠️ Crew kickoff failed: {e}")
        print("⚠️ Falling back to the HTTP-based Hugging Face inference workflow")
        result = run_research_workflow(RESEARCH_TOPIC, hf_token=HF_TOKEN, output_file='../research_summary.md')
else:
    print("⚠️ CrewAI not detected — using fallback run_research_workflow implementation")
    result = run_research_workflow(RESEARCH_TOPIC, hf_token=HF_TOKEN, output_file='../research_summary.md')

print("\n" + "="*60)
print("WORKFLOW EXECUTION")
print("="*60 + "\n")

🚀 CrewAI detected — constructing crew and launching kickoff...
⚠️ Crew kickoff failed: litellm.BadRequestError: HuggingfaceException - {"error":{"message":"The requested model 'mistralai/Mistral-7B-Instruct-v0.2' is not supported by any provider you have enabled.","type":"invalid_request_error","param":"model","code":"model_not_supported"}}
⚠️ Falling back to the HTTP-based Hugging Face inference workflow

Starting Multi-Agent Research Workflow
Topic: Impact of synthetic data in healthcare

⚠️ Crew kickoff failed: litellm.BadRequestError: HuggingfaceException - {"error":{"message":"The requested model 'mistralai/Mistral-7B-Instruct-v0.2' is not supported by any provider you have enabled.","type":"invalid_request_error","param":"model","code":"model_not_supported"}}
⚠️ Falling back to the HTTP-based Hugging Face inference workflow

Starting Multi-Agent Research Workflow
Topic: Impact of synthetic data in healthcare



ERROR:root:Crew kickoff failed: litellm.BadRequestError: HuggingfaceException - {"error":{"message":"The requested model 'mistralai/Mistral-7B-Instruct-v0.2' is not supported by any provider you have enabled.","type":"invalid_request_error","param":"model","code":"model_not_supported"}}
ERROR:crewai.events.listeners.tracing.trace_batch_manager:Failed to send events: 401. Response: {"error":"bad_credentials","message":"Bad credentials"}. Events will be lost.
ERROR:crewai.events.listeners.tracing.trace_batch_manager:Failed to send events: 401. Response: {"error":"bad_credentials","message":"Bad credentials"}. Events will be lost.
ERROR:crewai.events.listeners.tracing.trace_batch_manager:Failed to send events: 401. Response: {"error":"bad_credentials","message":"Bad credentials"}. Events will be lost.
ERROR:crewai.events.listeners.tracing.trace_batch_manager:Failed to send events: 401. Response: {"error":"bad_credentials","message":"Bad credentials"}. Events will be lost.
ERROR:root:Write


Research summary saved to: ../research_summary.md


WORKFLOW EXECUTION



In [156]:
# The workflow function performs web search, writes a draft, and runs a review loop

print("\n" + "="*60)
print("✓ Workflow completed successfully!")
print("="*60)


✓ Workflow completed successfully!


## View and Save Results

Let's examine the output from each agent and save the final summary:

In [157]:
# Display results from each task
print("\n" + "="*60)
print("TASK OUTPUTS")
print("="*60)

if hasattr(result, 'tasks_output') and result.tasks_output:
    for i, task_output in enumerate(result.tasks_output, 1):
        print(f"\n--- Task {i} Output ---")
        print(task_output.raw_output)
        print("\n")
else:
    print("\nFinal Result:")
    print(result)


TASK OUTPUTS

Final Result:
{'topic': 'Impact of synthetic data in healthcare', 'initial_searches': ['Apr 6, 2023 · In this paper, we present the cases for physical and statistical simulations for creating data and the proposed applications in healthcare and medicine. We discuss that while synthetics can promote privacy, equity, safety and continual and causal learning, they also run the risk of introducing flaws, blind spots and propagating or exaggerating ... Jun 19, 2025 · The review systematically examines biomedical research and application trends in synthetic data generation, emphasizing clinical applications, methodologies, and evaluations. lications in healthcare and medicine. We discuss that while synthetics can promote privacy, equity, safety and continual and causal learning, they also run the risk of introducing flaws, blind spots a Keywords Healthcare , Machine Learning, Synthetic Data Abstract Synthetic data are becoming a critical tool for building artificially intellig

In [158]:
# result is a dict returned by run_research_workflow with keys: draft_summary, review, final_summary
research_findings = result.get('initial_searches', [])
final_summary = result.get('final_summary', '')
review_feedback = result.get('review', {})

# Save to file
output_file = "../research_summary.md"
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(final_summary)

print(f"✓ Research summary saved to: {output_file}")

✓ Research summary saved to: ../research_summary.md


## Display Final Summary

Let's display the final research summary in a nicely formatted way:

## Review Feedback

Here's what the Reviewer Agent thought about the summary:

In [159]:
print("\n" + "="*60)
print("REVIEWER FEEDBACK")
print("="*60 + "\n")

if isinstance(review_feedback, dict):
    md = "### Reviewer Scores\n\n"
    scores = review_feedback.get('scores', {})
    for k, v in scores.items():
        md += f"- **{k.capitalize()}**: {v}/5\n"
    suggestions = review_feedback.get('suggestions', [])
    if suggestions:
        md += "\n### Suggestions\n\n"
        for s in suggestions:
            md += f"- {s}\n"
    else:
        md += "\n_No suggestions provided._\n"
    display(Markdown(md))
else:
    display(Markdown(str(review_feedback)))


REVIEWER FEEDBACK



### Reviewer Scores

- **Coherence**: 4/5
- **Factual_accuracy**: 3/5
- **Structure**: 5/5
- **Completeness**: 4/5
- **Clarity**: 4/5

_No suggestions provided._


## Alternative: Use the Simplified Function

You can also use the `run_research_workflow` function for a simpler one-line execution:

In [160]:
# Quick execution with a different topic
# Uncomment to try:

# new_topic = "Bias in Large Language Models"
# result = run_research_workflow(
#     topic=new_topic,
#     hf_token=HF_TOKEN,
#     output_file="../research_summary_bias.md"
# )

## Summary

This notebook demonstrated a complete multi-agent research workflow using:

### Technologies Used ✓
- **CrewAI** for multi-agent orchestration and communication cycles
- **LangChain** for tool integration (DuckDuckGo search)
- **Hugging Face Inference API** for all LLM operations (NO local LLMs)

### Hugging Face Models Used ✓
| Agent | Model | Purpose |
|-------|-------|---------|
| Researcher | `mistralai/Mistral-7B-Instruct-v0.2` | Reasoning and coordination |
| Writer | `HuggingFaceH4/zephyr-7b-beta` | Summarization |
| Reviewer | `mistralai/Mistral-7B-Instruct-v0.2` | Text analysis |

### Agent Communication Workflow ✓
The three agents successfully collaborated through defined communication cycles:
1. **Researcher** → performs web search → returns snippets to Writer
2. **Writer** → generates first draft using Hugging Face API → sends to Reviewer
3. **Reviewer** → critiques and provides feedback → returned for refinement
4. **Writer** → finalizes Markdown report based on feedback

### Final Deliverables ✓
- **research_summary.md**: 500-word structured summary with:
  - Introduction
  - Key Findings
  - Ethical & Technical Challenges
  - Conclusion
- **Reviewer feedback**: Coherence and factuality evaluation

### Next Steps
- Try different research topics (healthcare, education, climate, etc.)
- Experiment with different Hugging Face models
- Add more specialized agents for specific domains
- Integrate additional search tools (Tavily, etc.)